# نمونہ 01: اوپن اے آئی ایس ڈی کے ذریعے فوری چیٹ

یہ نوٹ بک مائیکروسافٹ فاؤنڈری لوکل کے ساتھ اوپن اے آئی ایس ڈی کے استعمال کے ذریعے فوری چیٹ کے تعاملات کو ظاہر کرتی ہے۔

## جائزہ

یہ مثال دکھاتی ہے:
- فاؤنڈری لوکل کے ساتھ اوپن اے آئی پائتھون ایس ڈی کے کا استعمال
- Azure OpenAI اور مقامی فاؤنڈری کنفیگریشنز کو سنبھالنا
- مناسب غلطی ہینڈلنگ اور بیک اپ حکمت عملیوں کا نفاذ
- سروس مینجمنٹ کے لیے FoundryLocalManager کا استعمال


## ضروریات

یقینی بنائیں کہ آپ کے پاس موجود ہیں:
1. Foundry Local انسٹال اور چل رہا ہو
2. ایک ماڈل لوڈ کیا ہوا ہو (مثال کے طور پر، `foundry model run phi-4-mini`)
3. مطلوبہ Python پیکجز انسٹال کیے ہوئے ہوں

### ڈیپینڈنسیز انسٹال کریں


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## مطلوبہ لائبریریاں درآمد کریں


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## ترتیب

اپنی ترتیب سیٹ کریں۔ آپ درج ذیل میں سے کسی ایک کا استعمال کر سکتے ہیں:
1. **Azure OpenAI** (کلاؤڈ پر مبنی)
2. **Foundry Local** (مقامی انفرنس)

### آپشن 1: Azure OpenAI ترتیب


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### فاؤنڈری مقامی تشکیل: آپشن 2


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## کلائنٹ کی ابتدا

اپنی کنفیگریشن کی بنیاد پر OpenAI کلائنٹ بنائیں:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## بنیادی چیٹ کی مثال

آئیے ایک سادہ چیٹ تعامل کو آزماتے ہیں:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## انٹرایکٹو چیٹ سیشن

مختلف قسم کے پرامپٹس آزمائیں تاکہ دیکھ سکیں ماڈل کیسے جواب دیتا ہے:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## اعلیٰ استعمال: اسٹریمنگ جوابات

طویل جوابات کے لیے، اسٹریمنگ بہتر صارف تجربہ فراہم کر سکتی ہے:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## خرابیوں کا ازالہ اور تشخیص

آئیے سروس کی حالت اور دستیاب ماڈلز کو چیک کرتے ہیں:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## کسٹم پرامپٹ ٹیسٹنگ

نیچے دیے گئے سیل کا استعمال کریں تاکہ آپ اپنے پرامپٹس کو آزما سکیں:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## خلاصہ

اس نوٹ بک میں درج ذیل چیزیں دکھائی گئیں:

1. **✅ کلائنٹ سیٹ اپ**: Foundry Local کے ساتھ OpenAI SDK کو ترتیب دینے کا طریقہ
2. **✅ بنیادی چیٹ**: سادہ درخواست-جواب تعاملات
3. **✅ اسٹریمنگ**: حقیقی وقت میں جواب کی تخلیق
4. **✅ غلطیوں کا انتظام**: مضبوط غلطی کا انتظام اور تشخیص
5. **✅ سروس کی صحت**: ماڈل کی دستیابی اور سروس کی حالت چیک کرنا

### اگلے اقدامات

- **نمونہ 02**: Azure OpenAI سپورٹ کے ساتھ ایڈوانس SDK انٹیگریشن
- **نمونہ 04**: Chainlit چیٹ ایپلیکیشن بنانا
- **نمونہ 05**: ملٹی ایجنٹ آرکسٹریشن سسٹمز
- **نمونہ 06**: ذہین ماڈل روٹنگ

### Foundry Local کے اہم فوائد

- 🔒 **پرائیویسی**: ڈیٹا آپ کے ڈیوائس سے باہر نہیں جاتا
- ⚡ **رفتار**: کم تاخیر کے ساتھ مقامی انفرنس
- 💰 **لاگت**: API استعمال کی کوئی قیمت نہیں
- 🔌 **آف لائن**: انٹرنیٹ کنکشن کے بغیر کام کرتا ہے
- 🛠️ **مطابقت**: OpenAI-مطابقت پذیر API
